In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0,1'

## Sample Functions (generated by gpt4)
> [ChatGPT Thread](https://chat.openai.com/share/6ed2d0bb-ec35-4273-85b8-113d37db7f43)

In [2]:
sample_functions = dict(
    personal_trainer={ "name": "logWeight", "description": "Logs the users weight and provides a visual representation of their weight change over time.", "parameters": { "type": "object", "properties": { "weight": { "type": "number" }, "date": { "type": "string", "format": "date" }, "notes": { "type": "string" } } } },
    budget_assistant={
        "name": "categorizeTransaction",
        "description": "This function categorizes transactions into budget categories based on the description provided.",
        "parameters": {
            "type": "object",
            "properties": {
                "transactionDescription": {
                    "type": "string"
                }
            }
        }
    },
    home_agent={
        "name": "adjustThermostat",
        "description": "Adjusts the home's thermostat to the desired temperature and mode.",
        "parameters": {
            "type": "object",
            "properties": {
                "temperature": {
                    "type": "number"
                },
                "mode": {
                    "type": "string"
                }
            }
        }
    },
    meal_planner={
        "name": "fetchRecipes",
        "description": "Search for recipes based on dietary preferences and available ingredients.",
        "parameters": {
            "type": "object",
            "properties": {
                "dietaryPreferences": {
                    "type": "string"
                },
                "availableIngredients": {
                    "type": "string"
                }
            }
        }
    },
    educational_tutor={
        "name": "generatePersonalizedQuiz",
        "description": "Creates a quiz tailored to the user's learning level and performance history in a specific subject.",
        "parameters": {
            "type": "object",
            "properties": {
                "UserID": {
                    "type": "string"
                },
                "Subject": {
                    "type": "string"
                },
                "DifficultyLevel": {
                    "type": "string"
                }
            }
        }
    },
    travel_planner={
        "name": "searchFlights",
        "description": "Searches for flights based on provided criteria (destination, departure date, return date, and budget).",
        "parameters": {
            "type": "object",
            "properties": {
                "destination": {
                    "type": "string"
                },
                "departureDate": {
                    "type": "string"
                },
                "returnDate": {
                    "type": "string"
                },
                "budget": {
                    "type": "number"
                }
            }
        }
    },
    meal_planner_2={
        "name": "findRecipesBasedOnIngredients",
        "description": "Searches a recipe database for recipes that can be made with a specific set of ingredients provided by the user.",
        "parameters": {
            "type": "object",
            "properties": {
                "ingredients": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    }
                }
            }
        }
    },
    travel_planner_2={
        "name": "findBestFlight",
        "description": "Finds the best flight options based on user preferences.",
        "parameters": {
            "type": "object",
            "properties": {
                "options": {
                    "type": "object",
                    "properties": {
                        "dates": {"type": "string"},
                        "destinations": {"type": "string"},
                        "budget": {"type": "number"}
                    }
                }
            }
        }
    },
    health_monitor={
        "name": "logHealthMetric",
        "description": "Logs various health metrics such as steps taken, heart rate, or sleep quality, along with the exact time of recording.",
        "parameters": {
            "type": "object",
            "properties": {
                "metricName": {
                    "type": "string"
                },
                "value": {
                    "type": "number"
                },
                "timestamp": {
                    "type": "string"
                }
            }
        }
    },
    ecommerce_assistant={
        "name": "findProduct",
        "description": "Searches for products based on a user's query and optional filters.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string"
                },
                "filters": {
                    "type": "object",
                    "properties": {
                        "priceRange": {
                            "type": "string"
                        },
                        "category": {
                            "type": "string"
                        }
                    }
                }
            }
        }
    },
)

## Process dataset

In [3]:
from datasets import load_dataset

ds = load_dataset("togethercomputer/glaive-function-calling-v2-formatted")

In [4]:
ds = ds.remove_columns("text")

In [5]:
import json
import random

def convert_tools_to_functions(row):
    tools = json.loads(row["tools"])

    # Get functions
    functions = (
        # [tool["function"] for tool in tools]
        # if tools else
        random.sample(list(sample_functions.values()), 1)
    )

    return dict(
        functions='[]', # json.dumps(functions),  # hf datasets cant hold arbitrary types
        use_function=False
    )

ds = ds.map(convert_tools_to_functions).remove_columns("tools")

In [6]:
def replace_system_message(row):
    situation_content = "You are a helpful assistant with access to one or more tools. Use them only if required to fulfill a user's request."
    messages = json.loads(row["messages"])

    # Sanity check
    assert messages[0]["role"] == "system"
    
    # Replace system message
    messages[0] = dict(
        role="system",
        name="situation",
        content=situation_content,
    )

    return dict(
        messages=messages[:2],  # Only keep system and user messages
    )

ds = ds.map(replace_system_message)

In [7]:
ds = ds.filter(lambda row: all(msg["content"] for msg in row["messages"]))

In [8]:
from model_api.conversion.conversions import to_prompt, parse_message
from model_api.conversion.datatypes import ChatMLMessage
from model_api.protocol import FunctionDef

# Convert to prompts
convert_to_prompt = lambda row: dict(
    prompt=to_prompt(
        messages=[
            ChatMLMessage(**message)
            for message in row["messages"]
        ],
        functions=[
            FunctionDef(**fn)
            for fn in json.loads(row["functions"])
        ],
    )
)

ds = ds.map(convert_to_prompt)

## Start engine

In [9]:
from vllm import AsyncLLMEngine, AsyncEngineArgs

engine_args = AsyncEngineArgs(
    model="julep-ai/samantha-1-turbo",
    dtype="bfloat16",
    enforce_eager=False,
    tensor_parallel_size=2,
    swap_space=4,  # GiB
    gpu_memory_utilization=0.98,
    max_num_seqs=256,
)


engine_args

AsyncEngineArgs(model='julep-ai/samantha-1-turbo', tokenizer='julep-ai/samantha-1-turbo', tokenizer_mode='auto', trust_remote_code=False, download_dir=None, load_format='auto', dtype='bfloat16', kv_cache_dtype='auto', seed=0, max_model_len=None, worker_use_ray=False, pipeline_parallel_size=1, tensor_parallel_size=2, max_parallel_loading_workers=None, block_size=16, swap_space=4, gpu_memory_utilization=0.98, max_num_batched_tokens=None, max_num_seqs=256, max_paddings=256, disable_log_stats=False, revision=None, tokenizer_revision=None, quantization=None, enforce_eager=False, max_context_len_to_capture=8192, disable_custom_all_reduce=False, enable_lora=False, max_loras=1, max_lora_rank=16, lora_extra_vocab_size=256, max_cpu_loras=None, engine_use_ray=False, disable_log_requests=False, max_log_len=None)

In [10]:
engine = AsyncLLMEngine.from_engine_args(engine_args)

2024-02-20 15:57:03,178	INFO worker.py:1724 -- Started a local Ray instance.


INFO 02-20 15:57:04 llm_engine.py:72] Initializing an LLM engine with config: model='julep-ai/samantha-1-turbo', tokenizer='julep-ai/samantha-1-turbo', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 02-20 15:57:10 custom_all_reduce.py:125] NVLink detection failed with message "Not Supported". This is normal if your machine has no NVLink equipped
(RayWorkerVllm pid=289234) INFO 02-20 15:57:10 custom_all_reduce.py:125] NVLink detection failed with message "Not Supported". This is normal if your machine has no NVLink equipped
INFO 02-20 15:57:12 weight_utils.py:164] Using model weights format ['*.bin']
(RayWorkerVllm pid=289234) INFO 02-20 15:57:12 weight_utils.py:164] Using model weights format ['*.bin']
INFO 02-20 15:57:29 llm_engine.py:322] # GPU blocks: 5169, # CPU blocks: 4096
INFO 02-20 15:57:30 model_runner.py:632] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-20 15:57:30 model_runner.py:636] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_

## Tokenize prompt

In [11]:
tokenizer = engine.engine.tokenizer.tokenizer

ds = ds.map(
    lambda row: dict(
        prompt_token_ids=tokenizer.encode(row["prompt"])
    )
)

# )["train"][0]["prompt_token_ids"]

Map:   0%|          | 0/111944 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Prepare generator

In [12]:
from uuid import uuid4
from vllm.sampling_params import SamplingParams

def prep_generator(
    prompt_token_ids,
    temperature=0,
    max_tokens=1,
    logits_processors=[],
    **sampling_kwargs,
):
    sampling_params = SamplingParams(
        temperature=temperature,
        max_tokens=max_tokens,
        logits_processors=logits_processors,
        **sampling_kwargs,
    )
    
    res_generator = engine.generate(
        sampling_params=sampling_params,
        request_id=uuid4(),
        prompt=None,
        prompt_token_ids=prompt_token_ids,
    )

    return res_generator

async def generate(
    prompt_token_ids,
    **sampling_kwargs,
):
    res_generator = prep_generator(prompt_token_ids, **sampling_kwargs)
    final_res = None

    async for res in res_generator:
        final_res = res
    
    return final_res

def generate_no_wait(
    prompt_token_ids,
    **sampling_kwargs,
):
    res_generator = prep_generator(prompt_token_ids, **sampling_kwargs)

    async def waiter():
        final_res = None
        
        async for res in res_generator:
            final_res = res
        
        return final_res

    return waiter()

## Prep logits processor

In [13]:
# List of tags 
allowed_tags = ["me", "function_call", "thought"]
disallowed_tags = ["situation", "person", "functions", "information"]
tags = allowed_tags + disallowed_tags

allowed_tag_token_ids = [
    tokenizer(tag, add_special_tokens=False)["input_ids"]
    for tag in allowed_tags
]

disallowed_tag_token_ids = [
    tokenizer(tag, add_special_tokens=False)["input_ids"]
    for tag in disallowed_tags
]

tag_token_ids = [
    tokenizer(tag, add_special_tokens=False)["input_ids"]
    for tag in tags
]

tag_id_map = {
    tag: tag_ids[0]
    for tag, tag_ids in zip(tags, tag_token_ids)
}

id_tag_map = {
    id: tag
    for tag, id in tag_id_map.items()
}

In [14]:
import torch

requests: dict[str, tuple[str, list[int], torch.Tensor]] = dict(
    positive=[],
    negative=[],
)

def get_lp(type, prompt):
    def processor(
        previously_generated_tokens,
        next_token_logits,
    ):
        assert len(previously_generated_tokens) == 0
        
        requests[type].append(
            (prompt, previously_generated_tokens, next_token_logits.cpu())
        )

        return next_token_logits

    return processor

def reset_requests():
    global requests
    requests = dict(
        positive=[],
        negative=[],
    )

## Run all examples

In [15]:
import asyncio
from tqdm.auto import tqdm
import logging

logging.disable(logging.CRITICAL)

reset_requests()
pending = []

max_len = 15_000

for i, row in enumerate(ds["train"].shuffle(seed=42)):
    if i >= max_len:
        break
        
    key = "positive" if row["use_function"] else "negative"
    prompt_token_ids = row["prompt_token_ids"]
    prompt = row["prompt"]
            
    logits_processors = [
        get_lp(key, prompt),
    ]
    
    pending.append(
        generate_no_wait(prompt_token_ids, logits_processors=logits_processors, max_tokens=1)
    )

completed = asyncio.as_completed(pending)

for future in tqdm(completed, total=max_len):
    await future

  0%|          | 0/15000 [00:00<?, ?it/s]

In [16]:
import pickle

# open a file, where you ant to store the data
with open('./processed_new_new.pickle', 'wb') as processed_file:

    # dump information to that file
    pickle.dump(requests, processed_file)